# Heurística para Orienteering Problem with Time Windows (OPTW)

Sacado del paper [Kantan & Rosenwein (1992)](https://www.tandfonline.com/doi/abs/10.1057/jors.1992.88)
Vamos a trabajar con la instancia `50_c101.txt` sacada de Righini and Salani: c-r-rc-100-50, disponible [aca](https://www.mech.kuleuven.be/en/cib/op#section-8). (La descripción de la instancia puede ser encontrada [aca](https://www.mech.kuleuven.be/en/cib/op/instances/TOPTWformat/view))

## Empezamos por la clásica lectura de datos

In [1]:
import numpy as np
import pandas as pd

In [2]:
ruta = '50_c101.txt'
archivo = pd.read_csv(ruta,header=None, delim_whitespace=True)
archivo.columns =['i', 'x','y','d','S', 'f', 'a','lista', 'O','C']
archivo.head(10)

,i,x,y,d,S,f,a,lista,O,C
0,0,40.0,50.0,0.0,0.0,0,0,0,0,1236
1,1,45.0,68.0,90.0,10.0,1,1,1,912,967
2,2,45.0,70.0,90.0,30.0,1,1,1,825,870
3,3,42.0,66.0,90.0,10.0,1,1,1,65,146
4,4,42.0,68.0,90.0,10.0,1,1,1,727,782
5,5,42.0,65.0,90.0,10.0,1,1,1,15,67
6,6,40.0,69.0,90.0,20.0,1,1,1,621,702
7,7,40.0,66.0,90.0,20.0,1,1,1,170,225
8,8,38.0,68.0,90.0,20.0,1,1,1,255,324
9,9,38.0,70.0,90.0,10.0,1,1,1,534,605


In [3]:
T_max = archivo['C'][0]
print(T_max)

1236


In [4]:
len(archivo)

51

Calculamos las distancias

In [5]:
num_nodos = len(archivo)
#num_nodos = 7 #Para probar con porciones más pequeñas
dist = np.empty((num_nodos,num_nodos), dtype = int)

for i in range(num_nodos):
    a = np.array([archivo['x'][i],archivo['y'][i]])
    for j in range(num_nodos):
        if i!=j:
            b = np.array([archivo['x'][j],archivo['y'][j]])
            dist[i,j] = int(np.linalg.norm(a-b))
        else:
            dist[i,j] = 99999

In [6]:
print(dist)

[[99999    18    20 ...    23    19    22]
 [   18 99999     2 ...    41    37    40]
 [   20     2 99999 ...    43    38    42]
 ...
 [   23    41    43 ... 99999     5     2]
 [   19    37    38 ...     5 99999     3]
 [   22    40    42 ...     2     3 99999]]


Ahora comencemos con la implementación de la heurística

In [7]:
recorrido = []
por_visitar = [i for i in range(num_nodos)]
b = np.zeros(num_nodos, dtype = int)

In [8]:
print("recorrido=",recorrido)
print("Por visitar=",por_visitar)
print("b=",b)

recorrido= []
Por visitar= [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
b= [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [9]:
#inicializamos
recorrido.append(0)
por_visitar.remove(0)
recorrido.append(num_nodos-1)
por_visitar.remove(num_nodos-1)
b[num_nodos-1] = max(b[0]+dist[0,num_nodos-1],archivo['O'][num_nodos-1])
sum_tiempo = dist[0,num_nodos-1]
Funcion_objetivo = archivo['S'][num_nodos-1]            

In [10]:
print("Funcion_objetivo= ",Funcion_objetivo)
print("recorrido=",recorrido)
print("Por visitar",por_visitar)
print("b=", b)
print("sum_tiempo=",sum_tiempo)

Funcion_objetivo=  10.0
recorrido= [0, 50]
Por visitar [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
b= [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0 815]
sum_tiempo= 22


In [11]:
#comenzamos con unas pruebas
iks = 2
while (iks != 0):
    print("\niteracion ", 2-iks)
    rho = -99999
    for j in range(len(por_visitar)):
        print("opciones para p(j=%d)=%d "%(j,por_visitar[j]), "en", por_visitar)
        for k in range(len(recorrido)-1):
            print("Check 1: j=%d,u=%d,v=%d"%(por_visitar[j],recorrido[k],recorrido[k+1]))
            print(dist[recorrido[k],por_visitar[j]],dist[por_visitar[j],recorrido[k+1]],dist[recorrido[k],recorrido[k+1]])
            dividendo = dist[recorrido[k],por_visitar[j]]+dist[por_visitar[j],recorrido[k+1]]-dist[recorrido[k],recorrido[k+1]]
            if dividendo == 0:
                print("combinacion:j=%d,u=%d,v=%d"%(por_visitar[j],recorrido[k],recorrido[k+1]))
                dividendo = 0.1
            if archivo['S'][por_visitar[j]]/dividendo > rho:
                rho = archivo['S'][por_visitar[j]]/dividendo
                print("Check 2: j=%d,u=%d,v=%d"%(por_visitar[j],recorrido[k],recorrido[k+1]),
                      "rho=",archivo['S'][por_visitar[j]]/dividendo)
    print("mejor rho en la serie:",rho)
    iks -= 1


iteracion  0
opciones para p(j=0)=1  en [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
Check 1: j=1,u=0,v=50
18 40 22
Check 2: j=1,u=0,v=50 rho= 0.2777777777777778
opciones para p(j=1)=2  en [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
Check 1: j=2,u=0,v=50
20 42 22
Check 2: j=2,u=0,v=50 rho= 0.75
opciones para p(j=2)=3  en [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
Check 1: j=3,u=0,v=50
16 37 22
opciones para p(j=3)=4  en [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,

In [12]:
#ahora el algoritmo completo
itera = 0
cortar = False
while (len(por_visitar) != 0):
    print("\n\n---Iteración %d--- " %(itera))
    print("Recorrido hasta ahora",recorrido)
    print("Tiempo de arribo al nodo i hasta ahora",b)
    rho = -99999
    for j in range(len(por_visitar)):
        print("\nopciones para p(j=%d)=%d "%(j,por_visitar[j]), "en", por_visitar)
        for k in range(len(recorrido)-1):
            print("Combinación a probar: j=%d,u=%d,v=%d"%(por_visitar[j],recorrido[k],recorrido[k+1]))
            print("Dividendo: t[%d][%d] + t[%d][%d] - t[%d][%d] = %d + %d - %d = %d"
                  %(recorrido[k],por_visitar[j],por_visitar[j],recorrido[k+1],recorrido[k],
                    recorrido[k+1],dist[recorrido[k],por_visitar[j]],dist[por_visitar[j],recorrido[k+1]],
                    dist[recorrido[k],recorrido[k+1]],dist[recorrido[k],por_visitar[j]]+dist[por_visitar[j],recorrido[k+1]]-dist[recorrido[k],recorrido[k+1]]))
            dividendo = dist[recorrido[k],por_visitar[j]]+dist[por_visitar[j],recorrido[k+1]]-dist[recorrido[k],recorrido[k+1]]
            if dividendo == 0:
                tiempo = dividendo
                print("Cuidado dividendo igual cero, se cambio a 0.1")
                dividendo = 0.1
            else:
                tiempo = dividendo
            if archivo['S'][por_visitar[j]]/dividendo > rho:
                if sum_tiempo + tiempo <= T_max:
                    if b[recorrido[k]] + dist[recorrido[k],por_visitar[j]] <= archivo['C'][por_visitar[j]]:
                        b_temp = list(b)
                        b_temp[por_visitar[j]] = max(b_temp[recorrido[k]] + dist[recorrido[k],por_visitar[j]],
                                                     archivo['O'][por_visitar[j]])
                        #print("b[j]=b[%d]=max{b[%d]+t[%d][%d],O[%d]}=%d"%(por_visitar[j],recorrido[k],recorrido[k],
                        #                                               por_visitar[j],por_visitar[j],
                        #                                               max(b_temp[recorrido[k]] + 
                        #                                                   dist[recorrido[k],por_visitar[j]],
                        #                                                   archivo['O'][por_visitar[j]])))
                        for i in range(len(recorrido[k+1:])):
                            if i == 0:
                                b_temp[recorrido[i+k+1]] = max(b_temp[por_visitar[j]]+dist[por_visitar[j],recorrido[i+k+1]],
                                                  archivo['O'][recorrido[i+k+1]])
                                print("Cambiando b_temp[%d]=%d" %(recorrido[i+k+1],b_temp[recorrido[i+k+1]]))
                            else: 
                                b_temp[recorrido[i+k+1]] = max(b_temp[recorrido[i+k+1]]+dist[recorrido[i+k],recorrido[i+k+1]],
                                                  archivo['O'][recorrido[i+k+1]])
                                print("Cambiando b_temp[%d]=%d" %(recorrido[i+k+1],b_temp[recorrido[i+k+1]]))
                            if b_temp[recorrido[i+k+1]] > archivo['C'][recorrido[i+k+1]]:
                                print("Incumple condición 3): b[%d]=%d > d[%d]=%d" 
                                      %(recorrido[i+k+1],b_temp[recorrido[i+k+1]],recorrido[i+k+1],archivo['C'][recorrido[i+k+1]]))
                                cortar = True
                                break
                        if cortar:
                            cortar = False
                            break
                        rho = archivo['S'][por_visitar[j]]/dividendo
                        aumento = archivo['S'][por_visitar[j]]
                        diferencia_tiempo = tiempo
                        print(b_temp)
                        b_best = list(b_temp)
                        to_delete = por_visitar[j]
                        position = k+1
                        print("Combinación encontrada, añadida a posibles soluciones")
                    else:
                        print("Se inclumple condición 2): b[u=%d] +t[u=%d,j=%d] > C[j=%d] => %d + %d > %d"
                             %(recorrido[k],recorrido[k],por_visitar[j],por_visitar[j],
                               b[recorrido[k]],dist[recorrido[k],por_visitar[j]],
                               archivo['C'][por_visitar[j]]))
            else:
                print("Solución es peor que mejor rho encontrado")
    if rho != -99999:
        print("mejor rho en la serie:",rho,"insertando",to_delete,"en posición",position,"del recorrido")
        recorrido.insert(position,to_delete)
        por_visitar.remove(to_delete)
        Funcion_objetivo += aumento
        print("Función objetivo actual:",Funcion_objetivo)
        sum_tiempo += diferencia_tiempo
        b = list(b_best)
        print("El recorrido queda como: ", recorrido)
    else:
        print("\n\n---No se encontro una combinación posible---")
        break
    itera += 1
print("\nSolución Final")
print("Con recorrido:",recorrido)
print("Con Funcion_objetivo= ",Funcion_objetivo)
print("Quedaron por visitar:", por_visitar)
print("Tiempo de arribo al nodo i =", b)
print("Tiempo total (sin considerar esperas)=",sum_tiempo)



---Iteración 0--- 
Recorrido hasta ahora [0, 50]
Tiempo de arribo al nodo i hasta ahora [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0 815]

opciones para p(j=0)=1  en [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
Combinación a probar: j=1,u=0,v=50
Dividendo: t[0][1] + t[1][50] - t[0][50] = 18 + 40 - 22 = 36
Cambiando b_temp[50]=952
Incumple condición 3): b[50]=952 > d[50]=880

opciones para p(j=1)=2  en [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
Combinación a probar: j=2,u=0,v=50
Dividendo: t[0][2] + t[2][50] - t[0][50] = 20 + 42 - 22 = 40
Cambiand

Solución es peor que mejor rho encontrado

opciones para p(j=35)=36  en [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49]
Combinación a probar: j=36,u=0,v=46
Dividendo: t[0][36] + t[36][46] - t[0][46] = 35 + 28 - 20 = 43
Solución es peor que mejor rho encontrado
Combinación a probar: j=36,u=46,v=50
Dividendo: t[46][36] + t[36][50] - t[46][50] = 28 + 24 - 4 = 48
Solución es peor que mejor rho encontrado

opciones para p(j=36)=37  en [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49]
Combinación a probar: j=37,u=0,v=46
Dividendo: t[0][37] + t[37][46] - t[0][46] = 39 + 29 - 20 = 48
Solución es peor que mejor rho encontrado
Combinación a probar: j=37,u=46,v=50
Dividendo: t[46][37] + t[37][50] - t[46][50] = 29 + 25 - 4 = 50
Solu

Solución es peor que mejor rho encontrado
Combinación a probar: j=26,u=46,v=48
Dividendo: t[46][26] + t[26][48] - t[46][48] = 23 + 25 - 2 = 46
Solución es peor que mejor rho encontrado
Combinación a probar: j=26,u=48,v=50
Dividendo: t[48][26] + t[26][50] - t[48][50] = 25 + 23 - 2 = 46
Solución es peor que mejor rho encontrado

opciones para p(j=26)=27  en [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 47, 49]
Combinación a probar: j=27,u=0,v=43
Dividendo: t[0][27] + t[27][43] - t[0][43] = 17 + 19 - 16 = 20
Solución es peor que mejor rho encontrado
Combinación a probar: j=27,u=43,v=46
Dividendo: t[43][27] + t[27][46] - t[43][46] = 19 + 21 - 4 = 36
Solución es peor que mejor rho encontrado
Combinación a probar: j=27,u=46,v=48
Dividendo: t[46][27] + t[27][48] - t[46][48] = 21 + 22 - 2 = 41
Solución es peor que mejor rho encontrado
Combinación a probar: j=27,u=48,v=50
Di

Cambiando b_temp[20]=935
Incumple condición 3): b[20]=935 > d[20]=73

opciones para p(j=1)=2  en [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 44, 45, 47, 49]
Combinación a probar: j=2,u=0,v=20
Dividendo: t[0][2] + t[2][20] - t[0][20] = 20 + 25 - 10 = 35
Cambiando b_temp[20]=850
Incumple condición 3): b[20]=850 > d[20]=73

opciones para p(j=2)=3  en [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 44, 45, 47, 49]
Combinación a probar: j=3,u=0,v=20
Dividendo: t[0][3] + t[3][20] - t[0][20] = 16 + 20 - 10 = 26
Cambiando b_temp[20]=85
Incumple condición 3): b[20]=85 > d[20]=73

opciones para p(j=3)=4  en [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 44, 45, 47, 49]
Combinación a p

Tiempo total (sin considerar esperas)= 45


# Próxima Semana

Recuerden para la próxima semana tener instalado Latex, si quiere una ayuda, aquí tiene un [link](https://www.youtube.com/watch?v=bt4W0hjgFEY) de un compatriota que se lo explica rebien. 

Para poder usar Latex en nuestros computadores necesitamos descargar dos cosas:

El compilador (Como en c++)
- Para Windows usamos [MiKTex](http://miktex.org/download)
- Para Ubuntu: Mediante la terminal instalas el paquete texlive-full 
                sudo apt-get install texlive-full
- Para OS X: Instalas [MacTex](https://tug.org/mactex/)

Editor de texto:
- Instalamos Texmaker que es compatible con los 3 sistemas operativos, aca el [link](http://www.xm1math.net/texmaker/download.html)

Tambien tenemos opciones online como:
- [Overleaf](https://www.overleaf.com)